In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import differential_evolution

In [2]:
#read data
d = pd.read_excel('C:/Users/cape159/Documents/pracovni/data_statistika/kopackuv_grant/michaci_pokus/DB_theory/Hasan_Jolanta/Hasan_Jolanta.xlsx')

print(d)

   Soil Status   time        G12     G12init       G13   G13init      CO212  \
0    PL      A    0.1  72.316839   97.706528  2.022145  2.732098   0.654271   
1    PL      A   12.0  14.037931   97.706528  0.392533  2.732098  22.145056   
2    PL      A   24.0   0.255235   97.706528  0.007137  2.732098  36.088532   
3    PL      A   48.0   0.230056   97.706528  0.006433  2.732098  50.294775   
4    PL      A   72.0   0.000000   97.706528  0.000000  2.732098  61.281278   
5    CT      A    0.1  64.039704   88.978401  1.790697  2.488040   0.525789   
6    CT      A   12.0  29.680197   88.978401  0.829926  2.488040  10.303113   
7    CT      A   24.0   5.314360   88.978401  0.148602  2.488040  20.795284   
8    CT      A   48.0   0.200626   88.978401  0.005610  2.488040  37.865375   
9    CT      A   72.0   0.000000   88.978401  0.000000  2.488040  44.461055   
10   PL      N    0.1  77.975327  105.349123  2.148062  2.902155   0.199901   
11   PL      N   12.0  74.368740  105.349123  2.0487

In [3]:
#define DB model
def DBmodel (y, t, pars):
    #define initial states
    R12=y[0];    S12=y[1];    G12=y[2];   DOC12=y[3];   CO212=y[4]
    R13=y[5];    S13=y[6];    G13=y[7];   DOC13=y[8];   CO213=y[9]
        
    #define parameters
    Acg=pars[0];   Vmaxg=pars[1];    Kmg=pars[2] 
    Ac=pars[3];    Vmax=pars[4];     Km=pars[5] 
    m0=pars[6];    f=pars[7];        Yu=pars[8]
        
    
    #G uptake rate total
    Cug_tot12=Vmaxg*(S12+S13)*G12/(G12+G13+Kmg)
    Cug_tot13=Vmaxg*(S12+S13)*G13/(G12+G13+Kmg)
    
    #DOC uptake rate total
    Cu_tot12=Vmax*(S12+S13)*DOC12/(DOC12+DOC13+Km)
    Cu_tot13=Vmax*(S12+S13)*DOC13/(DOC12+DOC13+Km)
    
    #maintnance rate total
    m_tot12=m0*S12
    m_tot13=m0*S13
        
    #reserves mobilization rate total
    an_tot12=f*R12-m_tot12
    an_tot13=f*R13-m_tot13
    
    #define 12C atm% scaling factors
    Ratm=R12/(R12+R13)
    Satm=S12/(S12+S13)
    Gatm=G12/(G12+G13)
    DOCatm=DOC12/(DOC12+DOC13)
    
    #respiration rate 12 C
    if an_tot12 > m_tot12:
        r12=m_tot12+np.maximum(an_tot12*(1-Yu), 0)+(1-Acg)*Cug_tot12+(1-Ac)*Cu_tot12
    else:
        r12=(m_tot12+an_tot12)*Satm+f*R12+(1-Acg)*Cug_tot12+(1-Ac)*Cu_tot12
             
    #respiration rate 13 C
    if an_tot13 > m_tot13:
        r13=m_tot13+np.maximum(an_tot13*(1-Yu), 0)+(1-Acg)*Cug_tot13+(1-Ac)*Cu_tot13
    else:
        r13=(m_tot13+an_tot13)*(1-Satm)+f*R13+(1-Acg)*Cug_tot13+(1-Ac)*Cu_tot13
                  
            
    #derivatives
    dR12dt=Acg*Cug_tot12+Ac*Cu_tot12-f*R12
    dS12dt=np.maximum(an_tot12*Yu, 0)+np.minimum(0, (an_tot12+an_tot13)*Satm)
    dG12dt=-Cug_tot12
    dDOC12dt=-Cu_tot12
    dCO212dt=r12
    dR13dt=Acg*Cug_tot13+Ac*Cu_tot13-f*R13
    dS13dt=np.maximum(an_tot13*Yu, 0)+np.minimum(0, (an_tot12+an_tot13)*(1-Satm))
    dG13dt=-Cug_tot13
    dDOC13dt=-Cu_tot13
    dCO213dt=r13
    
            
    return dR12dt, dS12dt, dG12dt, dDOC12dt, dCO212dt, dR13dt, dS13dt, dG13dt, dDOC13dt, dCO213dt;

In [4]:
#define a function returning ode results with additional calculations
def calc (model, pars, t, y0):
    #these are the model parameters
    pars1=pars[0:9]
    
    #these are the parameters to recalculate R and S to Cmic
    pars2=pars[9:11]
    
    #first solve the model
    y = odeint(model,y0,t, args=(pars1,))
    #y = pd.DataFrame(y)
    #y.columns = ['R', 'S', 'DOC', 'CO2']
    #Cu=pars1[1]*y[:, 1]*y[:, 2]/(y[:, 2]+pars1[2])
    
    #calculate respiration rates and add it to y frame
    #r = y[:, 1]*pars1[3]+np.maximum((pars1[4]*y[:, 0]-y[:, 1]*pars1[3])*(1-pars1[5]), 0)+Cu*(1-pars1[0])
    
    #calculate Cmic12 and add it to y frame
    Cmic12 = pars2[0] * y[:, 0] + pars2[1] * y[:, 1]
    #calculate Cmic13 and add it to y frame
    Cmic13 = pars2[0] * y[:, 5] + pars2[1] * y[:, 6]
    
    yhat = np.concatenate((y[:, 2].reshape(5,1),#G12
                           y[:, 3].reshape(5,1),#DOC12
                           y[:, 4].reshape(5,1),#CO212
                           y[:, 7].reshape(5,1),#G13
                           y[:, 8].reshape(5,1),#DOC13
                           y[:, 9].reshape(5,1),#CO213
                           Cmic12.reshape(5,1),
                           Cmic13.reshape(5,1)), axis=1)
    
    return yhat

In [5]:
#create the minimization function
def obj_fun (x):
    #define parameters
    pars = x
    
    #initial conditions
    Cmic12init = data.Cmic12init[0]
    G12init = data.G12init[0]
    DOC12init = data.DOC12init[0]
    Cmic13init = data.Cmic13init[0]
    G13init = data.G13init[0]
    DOC13init = data.DOC13init[0]
    R12init = pars[11]
    R13init = pars[11]*(Cmic13init/Cmic12init)
    S12init = (Cmic12init-R12init*pars[9])/pars[10]
    S13init = (Cmic13init-R13init*pars[9])/pars[10]
    
    
    y0 = np.array([R12init, S12init, G12init, DOC12init,0,
                   R13init, S13init, G13init, DOC13init,0])
    
    #times
    t = data.time
    
    #use the function to get DOC, respiration rate and Cmic
    yhat_full = calc(DBmodel, pars[0:11], t, y0)
         
    #observations
    obs=np.concatenate((np.array([data.G12]).reshape(5,1),
                        np.array([data.DOC12]).reshape(5,1),
                        np.array([data.CO212]).reshape(5,1),
                        np.array([data.G13]).reshape(5,1),
                        np.array([data.DOC13]).reshape(5,1),
                        np.array([data.CO213]).reshape(5,1),
                        np.array([data.Cmic12]).reshape(5,1),
                        np.array([data.Cmic13]).reshape(5,1)), 
                     axis=1)
    
    #weights
    weights=np.concatenate((np.nanmean(data.G12).repeat(5).reshape(5,1),
                            np.nanmean(data.DOC12).repeat(5).reshape(5,1),
                            np.nanmean(data.CO212).repeat(5).reshape(5,1),
                            np.nanmean(data.G13).repeat(5).reshape(5,1),
                            np.nanmean(data.DOC13).repeat(5).reshape(5,1),
                            np.nanmean(data.CO213).repeat(5).reshape(5,1),
                            np.nanmean(data.Cmic12).repeat(5).reshape(5,1),
                            np.nanmean(data.Cmic13).repeat(5).reshape(5,1)), 
                       axis=1)
                
          
    out=np.nansum(((yhat_full-obs)/weights)**2)
          
    return out

In [6]:
#Plesne aerobni
data = d[(d.Soil=='PL') & (d.Status=='A')]
data = data.reset_index(drop=True)
dataCmic = data.Cmic12init[0]

optimum_PA=differential_evolution(obj_fun, [(0, 1), (0.0001, 10), (0.1, 100), (0, 1), (0.0001, 10), (0.1, 100),
                                            (0.0001, 1), (0.0001, 10), (0, 1), 
                                            (0, 1), (0,1), 
                                            (dataCmic*0.01, dataCmic*0.99)], polish=True, maxiter=1000000)

print(optimum_PA)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:236: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


     fun: 4.3365636280162905
     jac: array([ 1.24063311,  1.77257595, -0.53343223,  0.34121133,  0.77290876,
       -0.13264509,  1.63291816,  0.8124732 ,  0.90339336,  0.61437406,
       -0.17066544,  0.37963961])
 message: 'Optimization terminated successfully.'
    nfev: 42564
     nit: 232
 success: True
       x: array([9.11106644e-01, 1.88018256e-01, 9.14261859e+01, 2.58342504e-02,
       5.12394162e-04, 4.59706073e+01, 1.15627437e-03, 6.59717865e-02,
       7.40957625e-01, 4.38322805e-01, 3.42751361e-01, 4.66253402e+01])


In [7]:
#Certovo aerobni
data = d[(d.Soil=='CT') & (d.Status=='A')]
data = data.reset_index(drop=True)
dataCmic = data.Cmic12init[0]

optimum_CA=differential_evolution(obj_fun, [(0, 1), (0.0001, 10), (0.1, 100), (0, 1), (0.0001, 10), (0.1, 100),
                                            (0.0001, 1), (0.0001, 10), (0, 1), 
                                            (0, 1), (0,1), 
                                            (dataCmic*0.01, dataCmic*0.99)], polish=True, maxiter=1000000)
print(optimum_CA)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:236: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-p

     fun: 3.8685539868272834
     jac: array([0.89149026, 8.17711019, 1.10923772, 0.91684202, 0.53107874,
       0.35322127, 8.31048159, 9.45648293, 0.14023112, 6.27666359,
       8.46077617, 0.99195439])
 message: 'Optimization terminated successfully.'
    nfev: 35118
     nit: 185
 success: True
       x: array([7.94198971e-01, 8.68589361e-02, 9.74347277e+01, 9.82038818e-01,
       1.11204125e-03, 8.84636672e+01, 2.23207148e-03, 5.29246146e-01,
       9.94464336e-01, 6.57686674e-01, 3.71948698e-01, 1.80474880e+01])


In [8]:
#Plesne anaerobni
data = d[(d.Soil=='PL') & (d.Status=='N')]
data = data.reset_index(drop=True)
dataCmic = data.Cmic12init[0]

optimum_PN=differential_evolution(obj_fun, [(0, 1), (0.0001, 10), (0.1, 100), (0, 1), (0.0001, 10), (0.1, 100),
                                            (0.0001, 1), (0.0001, 10), (0, 1), 
                                            (0, 1), (0,1), 
                                            (dataCmic*0.01, dataCmic*0.99)], polish=True, maxiter=1000000)

print(optimum_PN)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:236: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-p

     fun: 10.809353153664903
 message: 'Optimization terminated successfully.'
    nfev: 104493
     nit: 578
 success: True
       x: array([9.80542768e-01, 3.43347872e-03, 6.35302139e+01, 9.93486325e-01,
       7.25508203e+00, 4.90940605e+00, 1.31779130e-04, 8.17562965e+00,
       9.97794852e-01, 5.03788565e-03, 2.15121351e-01, 4.72881813e+01])


In [9]:
#Certovo anaerobni
data = d[(d.Soil=='CT') & (d.Status=='N')]
data = data.reset_index(drop=True)
dataCmic = data.Cmic12init[0]

optimum_CN=differential_evolution(obj_fun, [(0, 1), (0.0001, 10), (0.1, 100), (0, 1), (0.0001, 10), (0.1, 100),
                                            (0.0001, 1), (0.0001, 10), (0, 1), 
                                            (0, 1), (0,1), 
                                            (dataCmic*0.01, dataCmic*0.99)], polish=True, maxiter=1000000)

print(optimum_CN)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:236: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


     fun: 10.343011192324266
     jac: array([-4.27931468e-02,  1.13053389e-01,  1.85583104e-02,  1.03409192e-01,
        4.45644233e-01,  3.55271368e-07, -1.95028349e-01,  4.78995755e+01,
        0.00000000e+00, -1.29177558e+00, -1.75099046e-02,  1.31487710e-02])
 message: 'Optimization terminated successfully.'
    nfev: 53481
     nit: 292
 success: True
       x: array([9.08536048e-01, 1.92128259e-02, 1.84043214e+00, 1.90838136e-01,
       1.02494751e-03, 4.19549493e+01, 1.07895695e-02, 1.00000000e-04,
       3.56159879e-01, 1.00000000e+00, 8.73615240e-01, 1.03272863e+00])
